## 1. Введение. Знакомство с датасетом

⚡ В качестве итогового задания проекта вы подготовите текстовый Google-документ, в котором по каждой встречающейся вам в этом модуле задаче приведёте ваш ответ в формате:

номер задания;
код с пояснениями;
результат запроса (в виде таблицы);
общий вывод в конце по результатам анализа рынка труда.

## 2. Предварительный анализ данных

Задание 2.1
Рассчитайте максимальный возраст (max_age) кандидата в таблице.

select
    max(c.age) max_age
from hh.candidate c

max_age
100

Задание 2.2
Теперь давайте рассчитаем минимальный возраст (min_age) кандидата в таблице.

select
    min(c.age) min_age
from hh.candidate c

min_age
14

Задание 2.3
Попробуем «почистить» данные. Напишите запрос, который позволит посчитать, сколько (cnt) человек какого возраста (age) у нас есть.
Отсортируйте результат по возрасту в обратном порядке.

select
    c.age,
    count(c.id)
from hh.candidate c
group by 1
order by 1 desc

Вывод: выполнен подсчет кандидатов и их сортировка по возрасту. 100 среди значений возраста встречается только один раз, в то время как остальные данные похожи на правду. Значит, при необходимости анализировать возраст в дальнейших подсчётах лучше отбрасывать это число.

Задание 2.4
По данным Росстата, средний возраст занятых в экономике России составляет 39.7 лет. Мы округлим это значение до 40. Найдите количество кандидатов, которые старше данного возраста. Не забудьте отфильтровать «ошибочный» возраст 100.

select
    count(c.id) "Количество человек старше 40"
from hh.candidate c
where c.age != 100 and c.age>40

## 3. Глобальный анализ показателей

Задание 3.1
Для начала напишите запрос, который позволит узнать, сколько (cnt) у нас кандидатов из каждого города (city).
Отсортируйте результат по количеству в обратном порядке.

select 
    city_tbl.title city,
    count(cand_tbl.id) cnt
from hh.candidate cand_tbl
    join hh.city city_tbl on cand_tbl.city_id = city_tbl.id
group by 1
order by 2 desc

Задание 3.2
Москва бросается в глаза как, пожалуй, самый активный рынок труда. Напишите запрос, который позволит понять, каких кандидатов из Москвы устроит «проектная работа».
Формат выборки: gender, age, desirable_occupation, city, employment_type.
Отсортируйте результат по id кандидата.

select
    cand_data.gender,
    cand_data.age,
    cand_data.desirable_occupation,
    city_data.title city,
    cand_data.employment_type
from hh.candidate cand_data
    join hh.city city_data on cand_data.city_id = city_data.id
where city_data.title = 'Москва' and cand_data.employment_type like '%проектная работа%'
--group by 4,1
order by cand_data.id

Задание 3.3
Данных оказалось многовато. Отфильтруйте только самые популярные IT-профессии — разработчик, аналитик, программист.
Обратите внимание, что данные названия могут быть написаны как с большой, так и с маленькой буквы.
Отсортируйте результат по городу и id кандидата.

select
    cand_data.gender,
    cand_data.age,
    cand_data.desirable_occupation,
    city_data.title city,
    cand_data.employment_type
from hh.candidate cand_data
    join hh.city city_data on cand_data.city_id = city_data.id
where
    city_data.title = 'Москва'
    and cand_data.employment_type like '%проектная работа%'
    and (lower(cand_data.desirable_occupation) like '%азработчик%'
    or lower(cand_data.desirable_occupation) like '%налитик%'
    or lower(cand_data.desirable_occupation) like '%рограммист%')
order by
    city_data.title,
    cand_data.id

вариант с ilike
select
    cand_data.gender,
    cand_data.age,
    cand_data.desirable_occupation,
    city_data.title city,
    cand_data.employment_type
from hh.candidate cand_data
    join hh.city city_data on cand_data.city_id = city_data.id
where
    city_data.title = 'Москва'
    and cand_data.employment_type like '%проектная работа%'
    and (cand_data.desirable_occupation ilike '%разработчик%'
    or cand_data.desirable_occupation ilike '%аналитик%'
    or cand_data.desirable_occupation ilike '%программист%')
order by
    city_data.title,
    cand_data.id

Задание 3.4
Для общей информации попробуйте выбрать номера и города кандидатов, у которых занимаемая должность совпадает с желаемой. Формат выборки: id, city.
Отсортируйте результат по городу и id кандидата.

select
    cand_data.id,
    city_data.title city
from hh.candidate cand_data
    join hh.city city_data on cand_data.city_id = city_data.id
where
    cand_data.current_occupation = cand_data.desirable_occupation
order by
    city_data.title,
    cand_data.id

Задание 3.5
Определите количество кандидатов пенсионного возраста. Пенсионный возраст для мужчин наступает в 65 лет, для женщин — в 60 лет.

select
    count(*)
from hh.candidate cand_data
where
    cand_data.age != 100
    and cand_data.gender = 'M' and cand_data.age >= 65
    or cand_data.gender = 'F' and cand_data.age >= 60

## 4. Анализ кандидатов для заказчиков

Задание 4.1
Для добывающей компании нам необходимо подобрать кандидатов из Новосибирска, Омска, Томска и Тюмени, которые готовы работать вахтовым методом.
Формат выборки: gender, age, desirable_occupation, city, employment_type, timetable_type.
Отсортируйте результат по городу и номеру кандидата.

select
    cand_data.gender,
    cand_data.age,
    cand_data.desirable_occupation,
    city_data.title as city,
    cand_data.employment_type,
    tt.title timetable_type
from hh.candidate as cand_data
    join hh.city city_data on cand_data.city_id = city_data.id
    join hh.candidate_timetable_type as ctt on ctt.candidate_id = cand_data.id
    join hh.timetable_type as tt on tt.id = ctt.timetable_id
where
    (city_data.title in ('Новосибирск', 'Омск', 'Томск', 'Тюмень'))
    and tt.title = 'вахтовый метод'
order by
    city_data.title,
    cand_data.id


Задание 4.2
Для заказчиков из Санкт-Петербурга нам необходимо собрать список из 10 желаемых профессий кандидатов из того же города от 16 до 21 года (в выборку включается 16 и 21, сортировка производится по возрасту) с указанием их возраста, а также добавить строку Total с общим количеством таких кандидатов. 

(select
    cand_data.desirable_occupation,
    cand_data.age
from hh.candidate as cand_data
    join hh.city city_data on cand_data.city_id = city_data.id
where
    city_data.title = 'Санкт-Петербург'
    and age between 16 and 21
order by 2
limit 10)

union all

(select 
    'Total',
    count(*)
from hh.candidate as cand_data
    join hh.city city_data on cand_data.city_id = city_data.id
where
    city_data.title = 'Санкт-Петербург'
    and age between 16 and 21)